<a href="https://colab.research.google.com/github/sathsaraRasantha/Question_Answering_for_Sinhala_Language_using_Deep_NN_Architectures/blob/main/BiDAF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/sathsaraRasantha/Question_Answering_for_Sinhala_Language_using_Deep_NN_Architectures.git


Cloning into 'Question_Answering_for_Sinhala_Language_using_Deep_NN_Architectures'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
